In [1]:
# import sys
# !{sys.executable} -m pip install implicit

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [4]:
data = pd.read_csv('./data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [6]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

In [7]:
data_train.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [8]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


# Оценивание
За выполнени каждого задания 1 балл

4 балла -> отл

3 балла -> хор

И тд

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар?  
**Это условный товар**  
Зачем он нужен?   
**Если юзер не покупал товары из топ-5000, то он "купил" товар с id 999999**
Используя этот товар мы смещаем качество рекомендаций.
В какую сторону? 
***Улучшаем***  
Можно ли удалить этот товар?  
***Да**  
Уберите этот товар и сравните с качеством на семинаре.

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [9]:
data['weights'] = np.log(data['sales_value'])
data.loc[data['weights'] < 0] = 0
data

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,weights
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,0.329304
1,0,0,0,0,0,0.00,0,0.00,0,0,0.0,0.0,0.000000
2,0,0,0,0,0,0.00,0,0.00,0,0,0.0,0.0,0.000000
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0,0.190620
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,0.405465
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2396799,1613,41655820646,663,16102849,1,2.00,3262,-1.15,1231,95,0.0,0.0,0.693147
2396800,1001,41655829421,663,13217063,1,1.69,3131,0.00,2231,95,0.0,0.0,0.524729
2396801,1001,41655829421,663,13217800,1,1.69,3131,0.00,2231,95,0.0,0.0,0.524729
2396802,1167,41656790510,663,6410462,22451,43.98,3385,-0.65,1059,95,0.0,0.0,3.783735


In [10]:
items_weights = pd.DataFrame()
items_weights['item_id'] = data['item_id']
items_weights['weights'] = data['weights']
items_weights

,item_id,weights
0,1004906,0.329304
1,0,0.000000
2,0,0.000000
3,1082185,0.190620
4,8160430,0.405465
...,...,...
2396799,16102849,0.693147
2396800,13217063,0.524729
2396801,13217800,0.524729
2396802,6410462,3.783735


In [11]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    weights = 1*items_weights['weights']/items_weights['weights'].sum()
    # print(weights)
    items = items_weights['item_id']
    recs = np.random.choice(items, size=n, replace=False, p=weights)
    
    return recs.tolist()

Сделайте предсказания

In [12]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(10)

CPU times: user 3min 46s, sys: 29.1 s, total: 4min 15s
Wall time: 2min 11s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1120261, 17179856, 1106523, 1013321, 9553288]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[964329, 913210, 1015915, 5583751, 6773196]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[9396882, 1013597, 1049788, 9655679, 977798]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1011926, 5565304, 918772, 12172412, 12263277]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[893018, 6513713, 8011397, 939900, 892978]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[833556, 939431, 12301932, 6904472, 944257]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[9707240, 996961, 5566716, 988460, 6463826]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[1106409, 6534166, 6534178, 995242, 6552334]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[861899, 9296987, 1102275, 1038145, 12781982]"
9,16,"[1062973, 1082185, 13007710]","[1094174, 6534178, 1008750, 871741, 946995]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [13]:
result_all = pd.read_csv('./predictions/predictions_basic.csv') # закгрузка predict с семианара
result_all.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[1066066, 1051046, 12604677, 9420230, 10456539]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[1045288, 983646, 1029817, 7115539, 883268]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [14]:
result_all['weighted_random_recommendation'] = result['weighted_random_recommendation']
result_all.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,[ 821867 834484 856942 865456 889248 ...,"[1066066, 1051046, 12604677, 9420230, 10456539]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1120261, 17179856, 1106523, 1013321, 9553288]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[1045288, 983646, 1029817, 7115539, 883268]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[964329, 913210, 1015915, 5583751, 6773196]"


In [15]:
# your_code
def precision_at_k(recommended_list, bought_list, k):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    #print('RL', recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    #print(flags)
    
    precision = flags.sum() / len(recommended_list)
    # print(precision)
    
    return precision

In [16]:
result['precision'] = precision_at_k(result['weighted_random_recommendation'], result['actual'], 5)
result.head(10)

,user_id,actual,weighted_random_recommendation,precision
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1120261, 17179856, 1106523, 1013321, 9553288]",0.0
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[964329, 913210, 1015915, 5583751, 6773196]",0.0
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[9396882, 1013597, 1049788, 9655679, 977798]",0.0
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1011926, 5565304, 918772, 12172412, 12263277]",0.0
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[893018, 6513713, 8011397, 939900, 892978]",0.0
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[833556, 939431, 12301932, 6904472, 944257]",0.0
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[9707240, 996961, 5566716, 988460, 6463826]",0.0
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[1106409, 6534166, 6534178, 995242, 6552334]",0.0
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[861899, 9296987, 1102275, 1038145, 12781982]",0.0
9,16,"[1062973, 1082185, 13007710]","[1094174, 6534178, 1008750, 871741, 946995]",0.0


In [17]:
result['precision'].max()

0.0

In [18]:
result_popular_recommendation = pd.DataFrame()
result_popular_recommendation['actual'] = result_all['actual']
result_popular_recommendation['popular_recommendation'] = result_all['popular_recommendation']
result_popular_recommendation['precision'] = precision_at_k(result_popular_recommendation['popular_recommendation'], result_popular_recommendation['actual'], 5)
result_popular_recommendation.head(10)

,actual,popular_recommendation,precision
0,[ 821867 834484 856942 865456 889248 ...,"[6534178, 6533889, 1029743, 6534166, 1082185]",0.0
1,[ 835476 851057 872021 878302 879948 ...,"[6534178, 6533889, 1029743, 6534166, 1082185]",0.0
2,[ 920308 926804 946489 1006718 1017061 ...,"[6534178, 6533889, 1029743, 6534166, 1082185]",0.0
3,[ 840386 889774 898068 909714 929067 ...,"[6534178, 6533889, 1029743, 6534166, 1082185]",0.0
4,[ 835098 872137 910439 924610 992977 ...,"[6534178, 6533889, 1029743, 6534166, 1082185]",0.0
5,[ 864335 990865 1029743 9297474 10457112 ...,"[6534178, 6533889, 1029743, 6534166, 1082185]",0.0
6,[ 6534178 1104146 829197 840361 862070 ...,"[6534178, 6533889, 1029743, 6534166, 1082185]",0.0
7,[ 840601 867293 933067 951590 952408 ...,"[6534178, 6533889, 1029743, 6534166, 1082185]",0.0
8,[ 910439 1082185 959076 1023958 1082310 ...,"[6534178, 6533889, 1029743, 6534166, 1082185]",0.0
9,[ 1062973 1082185 13007710],"[6534178, 6533889, 1029743, 6534166, 1082185]",0.0


In [19]:
result_random_recommendation = pd.DataFrame()
result_random_recommendation['actual'] = result_all['actual']
result_random_recommendation['random_recommendation'] = result_all['random_recommendation']
result_random_recommendation['precision'] = precision_at_k(result_random_recommendation['random_recommendation'], result_random_recommendation['actual'], 5)
result_random_recommendation.head(10)

,actual,random_recommendation,precision
0,[ 821867 834484 856942 865456 889248 ...,"[1066066, 1051046, 12604677, 9420230, 10456539]",0.0
1,[ 835476 851057 872021 878302 879948 ...,"[1045288, 983646, 1029817, 7115539, 883268]",0.0
2,[ 920308 926804 946489 1006718 1017061 ...,"[1078385, 1080867, 1055362, 986192, 1101932]",0.0
3,[ 840386 889774 898068 909714 929067 ...,"[828229, 9803579, 1052439, 1062152, 834664]",0.0
4,[ 835098 872137 910439 924610 992977 ...,"[942585, 13095667, 1353699, 970326, 887275]",0.0
5,[ 864335 990865 1029743 9297474 10457112 ...,"[12385263, 1128334, 970383, 1060471, 1038484]",0.0
6,[ 6534178 1104146 829197 840361 862070 ...,"[928052, 1045114, 1112754, 1504123, 1028715]",0.0
7,[ 840601 867293 933067 951590 952408 ...,"[1420173, 1061529, 954328, 1837966, 880553]",0.0
8,[ 910439 1082185 959076 1023958 1082310 ...,"[6772821, 1770400, 8068807, 883762, 7129338]",0.0
9,[ 1062973 1082185 13007710],"[9526745, 999121, 826156, 15926118, 12183316]",0.0


In [20]:
result_random_recommendation['precision'].max()

0.0

In [21]:
result_itemitem = pd.DataFrame()
result_itemitem['actual'] = result_all['actual']
result_itemitem['itemitem'] = result_all['itemitem']
result_itemitem['precision'] = precision_at_k(result_itemitem['itemitem'], result_itemitem['actual'], 5)
result_itemitem.head(10)

,actual,itemitem,precision
0,[ 821867 834484 856942 865456 889248 ...,"[999999, 1082185, 981760, 1127831, 995242]",0.0
1,[ 835476 851057 872021 878302 879948 ...,"[999999, 1082185, 981760, 1098066, 995242]",0.0
2,[ 920308 926804 946489 1006718 1017061 ...,"[999999, 1082185, 981760, 1127831, 995242]",0.0
3,[ 840386 889774 898068 909714 929067 ...,"[999999, 1082185, 981760, 1127831, 995242]",0.0
4,[ 835098 872137 910439 924610 992977 ...,"[999999, 1082185, 981760, 1127831, 1098066]",0.0
5,[ 864335 990865 1029743 9297474 10457112 ...,"[999999, 1082185, 981760, 995242, 1098066]",0.0
6,[ 6534178 1104146 829197 840361 862070 ...,"[999999, 1082185, 981760, 1098066, 1127831]",0.0
7,[ 840601 867293 933067 951590 952408 ...,"[999999, 1082185, 981760, 1098066, 995242]",0.0
8,[ 910439 1082185 959076 1023958 1082310 ...,"[999999, 1082185, 981760, 1127831, 995242]",0.0
9,[ 1062973 1082185 13007710],"[999999, 1082185, 981760, 995242, 1029743]",0.0


In [22]:
result_itemitem['precision'].max()

0.0

In [23]:
result_cosine = pd.DataFrame()
result_cosine['actual'] = result_all['actual']
result_cosine['cosine'] = result_all['cosine']
result_cosine['precision'] = precision_at_k(result_cosine['cosine'], result_cosine['actual'], 5)
result_cosine.head(10)

,actual,cosine,precision
0,[ 821867 834484 856942 865456 889248 ...,"[1082185, 999999, 981760, 1127831, 1098066]",0.0
1,[ 835476 851057 872021 878302 879948 ...,"[1082185, 1098066, 981760, 999999, 826249]",0.0
2,[ 920308 926804 946489 1006718 1017061 ...,"[1082185, 999999, 981760, 1127831, 1098066]",0.0
3,[ 840386 889774 898068 909714 929067 ...,"[1082185, 981760, 999999, 1127831, 961554]",0.0
4,[ 835098 872137 910439 924610 992977 ...,"[1082185, 981760, 999999, 1098066, 826249]",0.0
5,[ 864335 990865 1029743 9297474 10457112 ...,"[1082185, 999999, 981760, 1098066, 1127831]",0.0
6,[ 6534178 1104146 829197 840361 862070 ...,"[1082185, 999999, 981760, 1098066, 1127831]",0.0
7,[ 840601 867293 933067 951590 952408 ...,"[1082185, 981760, 999999, 1098066, 826249]",0.0
8,[ 910439 1082185 959076 1023958 1082310 ...,"[1082185, 999999, 981760, 1127831, 878996]",0.0
9,[ 1062973 1082185 13007710],"[1082185, 981760, 999999, 866227, 12263788]",0.0


In [24]:
result_cosine['precision'].max()

0.0

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


Безлайны не улучшаться на топ-5000 лучших товаров, потому что:  
если 2 юзера оба купили очень популярный товар, то это еще не значит,что они похожи, но
если 2 юзера оба купили редкий товар, то они похожи

In [25]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Заведем фиктивный item_id (если юзер НЕ покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [28]:
# your_code
# %%time

model = ItemItemRecommender(K=3, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

In [29]:
[id_to_itemid[rec[0]] for rec in recs]

[999999, 1082185, 981760, 995242, 1029743]

In [30]:
result_itemitem_3 = pd.DataFrame()
result_itemitem_3['user_id'] = result_all['user_id']
result_itemitem_3['actual'] = result_all['actual']
result_itemitem_3.head(10)

,user_id,actual
0,1,[ 821867 834484 856942 865456 889248 ...
1,3,[ 835476 851057 872021 878302 879948 ...
2,6,[ 920308 926804 946489 1006718 1017061 ...
3,7,[ 840386 889774 898068 909714 929067 ...
4,8,[ 835098 872137 910439 924610 992977 ...
5,9,[ 864335 990865 1029743 9297474 10457112 ...
6,13,[ 6534178 1104146 829197 840361 862070 ...
7,14,[ 840601 867293 933067 951590 952408 ...
8,15,[ 910439 1082185 959076 1023958 1082310 ...
9,16,[ 1062973 1082185 13007710]


In [31]:
%%time

result_itemitem_3['itemitem_3'] = result_itemitem_3['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 96.4 ms, sys: 29 ms, total: 125 ms
Wall time: 154 ms


In [32]:
result_itemitem_3.head(10)

,user_id,actual,itemitem_3
0,1,[ 821867 834484 856942 865456 889248 ...,"[999999, 1082185, 981760, 995242, 1029743]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[999999, 1082185, 981760, 1098066, 6534178]"
2,6,[ 920308 926804 946489 1006718 1017061 ...,"[999999, 1082185, 981760, 995242, 1029743]"
3,7,[ 840386 889774 898068 909714 929067 ...,"[999999, 1082185, 981760, 995242, 1029743]"
4,8,[ 835098 872137 910439 924610 992977 ...,"[999999, 1082185, 981760, 6534178, 995242]"
5,9,[ 864335 990865 1029743 9297474 10457112 ...,"[999999, 1082185, 981760, 995242, 1029743]"
6,13,[ 6534178 1104146 829197 840361 862070 ...,"[999999, 1082185, 981760, 995242, 1029743]"
7,14,[ 840601 867293 933067 951590 952408 ...,"[999999, 1082185, 981760, 6534178, 995242]"
8,15,[ 910439 1082185 959076 1023958 1082310 ...,"[999999, 1082185, 981760, 995242, 1029743]"
9,16,[ 1062973 1082185 13007710],"[999999, 1082185, 981760, 866227, 12263788]"


In [35]:
result_itemitem_3['precision'] = precision_at_k(result_itemitem_3['itemitem_3'], result_itemitem_3['actual'], 5)
result_itemitem_3.head(10)

,user_id,actual,itemitem_3,precision
0,1,[ 821867 834484 856942 865456 889248 ...,"[999999, 1082185, 981760, 995242, 1029743]",0.0
1,3,[ 835476 851057 872021 878302 879948 ...,"[999999, 1082185, 981760, 1098066, 6534178]",0.0
2,6,[ 920308 926804 946489 1006718 1017061 ...,"[999999, 1082185, 981760, 995242, 1029743]",0.0
3,7,[ 840386 889774 898068 909714 929067 ...,"[999999, 1082185, 981760, 995242, 1029743]",0.0
4,8,[ 835098 872137 910439 924610 992977 ...,"[999999, 1082185, 981760, 6534178, 995242]",0.0
5,9,[ 864335 990865 1029743 9297474 10457112 ...,"[999999, 1082185, 981760, 995242, 1029743]",0.0
6,13,[ 6534178 1104146 829197 840361 862070 ...,"[999999, 1082185, 981760, 995242, 1029743]",0.0
7,14,[ 840601 867293 933067 951590 952408 ...,"[999999, 1082185, 981760, 6534178, 995242]",0.0
8,15,[ 910439 1082185 959076 1023958 1082310 ...,"[999999, 1082185, 981760, 995242, 1029743]",0.0
9,16,[ 1062973 1082185 13007710],"[999999, 1082185, 981760, 866227, 12263788]",0.0


In [37]:
result_itemitem_3['precision'].max()

0.0

### Не получилось не одного удачного предсказания!!!

### Задание 4. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.